In [ ]:
import time,json,os
import numpy as np
import pandas as pd
from inference import *
from common import preprocess,get_csv_files,g_analyse_path,g_ds_sample_path,g_model_path
from warnings import filterwarnings
filterwarnings("ignore") 
np.set_printoptions(suppress=True)
pd.set_option('display.float_format',lambda x : '%.8f' % x)

In [ ]:
os.system("mkdir -p %s"%g_analyse_path)
csv_list = []
get_csv_files(g_ds_sample_path,csv_list)
csv_list.sort()
all_models = get_all_models()

In [ ]:
all_result = []
for item in json.loads(all_models.to_json(orient="records")):
    model_name = item['model_name']
    dataset = item['dataset']
    model_file = item['model_file']
    if model_file.find("PSVM_model_python") >= 0:
        continue
    print(model_name,dataset)
    perf = CJPerformance(model_file)
    for csv in csv_list:
        valid_set = os.path.splitext(os.path.basename(csv))[0]
        print("inference ","%s(%s)"%(model_name,dataset), "validation set %s"%valid_set)
        result = perf.Test(csv,True)
        result['model'] = model_name
        result['train'] = dataset
        result['valid'] = valid_set
        print(result)
        all_result.append(result)
df_result = pd.DataFrame(all_result)

In [ ]:
df_result.to_csv("%svalidation.csv"%g_analyse_path)
df_result